<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_Assignment1/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
!wandb login

In [1]:
import keras
from keras.datasets import fashion_mnist
from PIL import Image
#import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from tensorflow.keras import initializers
import tensorflow as tf

#wandb.init(project="assignment-1", entity="swathi")
#classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

In [2]:
def load_data():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  X_train = np.transpose((np.array(x_train[:54000],dtype=int).reshape([54000,784]))/255)
  Y_train = np.transpose((np.array(y_train[:54000],dtype=int).reshape([54000,1])))
  X_validation = np.transpose((np.array(x_train[54000:],dtype=int).reshape([6000,784]))/255)
  Y_validation = np.transpose(np.array(y_train[54000:],dtype=int).reshape([6000,1]))
  X_test = np.transpose((np.array(x_test,dtype=int).reshape([10000,784]))/255)
  Y_test = np.transpose(np.array(y_test,dtype=int).reshape([10000,1]))
  return X_train, Y_train, X_validation, Y_validation, X_test, Y_test

In [ ]:
def log_images():
  images = []
  labels = []
  for i in range(len(X_train)):
    if len(labels)==len(classes):
      break
    j = random.randint(0,53999)
    if classes[Y_train[0,j]] not in labels:
      images.append(X_train[:,j].reshape([28,28]))
      labels.append(classes[Y_train[0,j]])
  wandb.log({"Examples": [wandb.Image(img, caption=caption) for img,caption in zip(images,labels)]})

Loading data

In [51]:
X_train, Y_train, X_validation, Y_validation, X_test, Y_test = load_data()

In [ ]:
log_images()

Defining basic parameters

In [4]:
N= [0,64,32,16,10]  #Insert the number of neurons in each layer
N[0]=X_train.shape[0]
l = len(N) - 1

Weights initialising

In [126]:
def initialize_weights_random(N):
  np.random.seed(1)
  W = []
  b = []
  W.append(0)
  b.append(0)
  for i in range(1,l+1):
    W.append((np.random.randn(N[i],N[i-1])))
    b.append(np.zeros((N[i],1)))

  return W,b 

In [6]:
def initialize_weights_xavier(N):
  initializer = tf.keras.initializers.GlorotNormal()
  W = []
  b = []
  W.append(0)
  b.append(0)
  for i in range(1,l+1):
    values = initializer(shape=(N[i],N[i-1]))
    W.append(np.array(values))
    b.append(np.zeros((N[i],1)))
    del values
  return W,b 

In [8]:
def initialize_weights(N,init):
  if init=='random':
    return initialize_weights_random(N)

  if init=='xavier':
    return initialize_weights_xavier(N)

In [9]:
def one_hot(X,y):
  one_hot_y=np.zeros((10,X.shape[1]))
  for i in range(X.shape[1]):
    one_hot_y[y[0,i],i] = 1
  
  return one_hot_y

Softmax

In [10]:
def softmax(a):
  return np.exp(a)/sum(np.exp(a))

Sigmoid

In [11]:
def sigmoid(a):
  return 1/(1+np.exp(-a))

Sigmoid derivative

In [12]:
def dSigmoid(a):
  return sigmoid(a)*(1-sigmoid(a))

Tanh activation function

In [13]:
def tanh(a):
  return np.tanh(a)

Derivative of the tanh function

In [14]:
def dTanh(a):
  return 1- np.square(tanh(a))

ReLU

In [31]:
def ReLU(a):
  return np.maximum(0,a)

Derivative of ReLU

In [35]:
def dReLU(a):
  return np.greater(a, 0).astype(int)

In [33]:
def activation(a,act):
  if act=='sigmoid':
    return sigmoid(a)
  if act=='tanh':
    return tanh(a)
  if act=='relu':
    return ReLU(a)

def dActivation(a,act):
  if act=='sigmoid':
    return dSigmoid(a)
  if act=='tanh':
    return dTanh(a)
  if act=='relu':
    return dReLU(a)

Cross Entropy

In [19]:
def calculate_loss(y_hat,Y):
  loss = 0
  for i in range(Y.shape[1]):
    loss -= np.log(y_hat[Y[0,i],i])
  print (loss)

Feed forward network

In [91]:
def feedforward(X_input, W, b, N, l, act):
  a=[]
  a.append(0)
  h=[]
  h.append(X_input)
  for i in range(1,l):
    a_new=b[i]+np.matmul(W[i],h[i-1])
    #print(a_new)
    h_new=activation(a_new,act)
    #print(h_new)
    a.append(a_new)
    h.append(h_new)

  a_new=b[l]+np.matmul(W[l],h[l-1])
  y_hat=softmax(a_new)
  a.append(a_new)

  return a,h,y_hat

In [71]:
W_temp,b_temp=initialize_weights(N,'random')
a_temp,h_temp,y_temp=feedforward(X_train, W_temp, b_temp, N, 3, 'sigmoid')

[[  9.8150452   22.44241148   4.13113424 ...  14.93775001  -1.28086221
   18.51525485]
 [ 23.84212258  18.2932436   15.68047188 ...  10.85931558  20.39132258
   16.98514119]
 [ 20.09030921   1.73745592   2.86693077 ...   8.7358468   10.59914442
   14.42851109]
 ...
 [  7.63659731  18.46648141   4.98559698 ...  11.05679314  19.17461883
   15.49825702]
 [ -2.27027782 -31.42893539  -3.63226847 ...   4.24305037 -20.48036024
  -22.1051447 ]
 [ 12.11588366  21.77799296   8.40384287 ...  17.56706834  17.6357907
   24.4015618 ]]
[[9.99945379e-01 1.00000000e+00 9.84189345e-01 ... 9.99999674e-01
  2.17403491e-01 9.99999991e-01]
 [1.00000000e+00 9.99999989e-01 9.99999845e-01 ... 9.99980776e-01
  9.99999999e-01 9.99999958e-01]
 [9.99999998e-01 8.50363632e-01 9.46187286e-01 ... 9.99839306e-01
  9.99975063e-01 9.99999458e-01]
 ...
 [9.99517765e-01 9.99999990e-01 9.93210713e-01 ... 9.99984221e-01
  9.99999995e-01 9.99999814e-01]
 [9.36146366e-02 2.24174799e-14 2.57742162e-02 ... 9.85839684e-01
  1.27

Backpropogation

In [129]:
def backrop(X,y,y_hat,W,a,h,act):
  grad_W=[]
  grad_b=[]
  grad_a=[]
  grad_h=[]
  
  da = y_hat - one_hot(X,y)
  grad_a.append(da)

  for i in reversed(range(1,l+1)):
    dw = np.matmul(da,np.transpose(h[i-1]))/(X.shape[1])
    db = np.sum(da, axis=1, keepdims=True)/(X.shape[1])
    dh_prev = np.matmul(np.transpose(W[i]),da)
    da_prev = dh_prev*dActivation(a[i-1],act)
    
    grad_W.append(dw)
    grad_b.append(db)
    grad_a.append(da_prev)
    grad_h.append(dh_prev)
    da=da_prev

  grad_W.append(0)
  grad_b.append(0)
  grad_W.reverse()
  grad_b.reverse()
  grad_a.reverse()
  grad_h.reverse()
  

  return grad_W,grad_b,grad_a,grad_h


Gradient Descent

In [22]:
def gradient_descent(epochs, learn_rate, batch_size, act, init, N):
  W, b = initialize_weights(N,init)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      a, h, y_hat = feedforward(X_train, W, b, N, l,act)
      if j==n-1:
        calculate_loss(y_hat, Y)
        wandb.log({"loss": avg_loss, "epoch": epoch})
      grad_W, grad_b = backrop(Y_train, y_hat, W, a, h, act)[0:2]
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for j in range(1,l+1):
        W_new.append(W[j] - (learn_rate*grad_W[j])/batch_size)
        b_new.append(b[j] - (learn_rate*grad_b[j])/batch_size)
      W = W_new
      b = b_new
  return W, b

Momentum based gradient descent

In [104]:
def mom_grad_descent(epochs, learn_rate, gamma, batch_size, act, init, N):
  W, b = initialize_weights(N,init)
  update_prev_w = []
  update_prev_w.append(0)
  update_prev_b = []
  update_prev_b.append(0)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(1,l+1):
    update_prev_w.append(np.zeros(W[i].shape))
    update_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      update_w = []
      update_w.append(0)
      update_b = []
      update_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l,act)
      if j==n-1:
        calculate_loss(y_hat, Y)
        #wandb.log({"loss": avg_loss, "epoch": epoch})
      grad_W, grad_b = backrop(X, Y, y_hat, W, a, h, act)[0:2]
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      
      for j in range(1,l+1):
        update_w.append(np.zeros(W[j].shape))
        update_b.append(np.zeros(b[j].shape))
        update_w[j] = gamma*update_prev_w[j] + learn_rate*grad_W[j]
        update_b[j] = gamma*update_prev_b[j] + learn_rate*grad_b[j]
        W_new.append((W[j] - update_w[j]/batch_size))
        b_new.append((b[j] - update_b[j]/batch_size))
        update_prev_w[j]=update_w[j]
        update_prev_b[j]=update_b[j]
      del update_w
      del update_b
      W = W_new
      b = b_new
    
  return W, b

In [ ]:
W, b = mom_grad_descent(10,0.001,0.9,1028,,'random',N)

Nesterov accelerated gradient descent

In [102]:
def nesterov_grad_descent(epochs, learn_rate, gamma, batch_size, act, init, N):
  W, b = initialize_weights(N,init)
  update_prev_w = []
  update_prev_w.append(0)
  update_prev_b = []
  update_prev_b.append(0)
  n=(np.floor(X_train.shape[1]/batch_size)+1).astype(int)
  for i in range(1,l+1):
    update_prev_w.append(np.zeros(W[i].shape))
    update_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      update_w = []
      update_w.append(0)
      update_b = []
      update_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      if j==n-1:
        calculate_loss(y_hat, Y)
        #wandb.log({"loss": avg_loss, "epoch": epoch})
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for j in range(1,l+1):
        update_w.append(np.zeros(W[j].shape))
        update_b.append(np.zeros(b[j].shape))
        update_w[j] = gamma*update_prev_w[j]
        update_b[j] = gamma*update_prev_b[j]
        W_new.append((W[j] - update_w[j]/batch_size))
        b_new.append((b[j] - update_b[j]/batch_size))
      W = W_new
      b = b_new
      grad_W, grad_b = backrop(X, Y, y_hat, W, a, h, act)[0:2]
      for j in range(1,l+1):
        update_w[j] = gamma*update_prev_w[j] + learn_rate*grad_W[j]
        update_b[j] = gamma*update_prev_b[j] + learn_rate*grad_b[j]
        W_new.append((W[j] - update_w[j]/batch_size))
        b_new.append((b[j] - update_b[j]/batch_size))
        update_prev_w[j]=update_w[j]
        update_prev_b[j]=update_b[j]
      del update_w
      del update_b
      W = W_new
      b = b_new
  return W, b

RMS Prop

In [55]:
def rms_prop(epochs, learn_rate, beta, epsilon, batch_size, act, init, N):
  W, b = initialize_weights(N,init)
  update_prev_w = []
  update_prev_w.append(0)
  update_prev_b = []
  update_prev_b.append(0)
  n=(np.floor(X_train.shape[1]/batch_size)+1).astype(int)
  for i in range(1,l+1):
    update_prev_w.append(np.zeros(W[i].shape))
    update_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      update_w = []
      update_w.append(0)
      update_b = []
      update_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      if j==n-1:
        print(a[1],h[1])
        calculate_loss(y_hat, Y)
        wandb.log({"loss": avg_loss, "epoch": epoch})
      grad_W, grad_b = backrop(X, Y, y_hat, W, a, h, act)[0:2]
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for j in range(1,l+1):
        update_w.append(np.zeros(W[j].shape))
        update_b.append(np.zeros(b[j].shape))
        update_w[j] = beta*update_prev_w[j] + (1-beta)*np.square(grad_W[j])
        update_b[j] = beta*update_prev_b[j] + (1-beta)*np.square(grad_b[j])
        W_new.append((W[j] - (learn_rate*np.divide(grad_W[j],(np.sqrt(update_w[j]+epsilon))))))
        b_new.append((b[j] - (learn_rate*np.divide(grad_b[j],(np.sqrt(update_b[j]+epsilon))))))
        update_prev_w[j]=update_w[j]
        update_prev_b[j]=update_b[j]
      del update_w
      del update_b
      W = W_new
      b = b_new
  return W, b

In [144]:
def adam(epochs, learn_rate, beta1, beta2, epsilon, batch_size, act, init, N):
  W, b = initialize_weights(N,init)
  m_prev_w = []
  m_prev_w.append(0)
  m_prev_b = []
  m_prev_b.append(0)
  v_prev_w = []
  v_prev_w.append(0)
  v_prev_b = []
  v_prev_b.append(0)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(1,l+1):
    m_prev_w.append(np.zeros(W[i].shape))
    m_prev_b.append(np.zeros(b[i].shape))
    v_prev_w.append(np.zeros(W[i].shape))
    v_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      m_w = []
      m_w.append(0)
      v_w = []
      v_w.append(0)
      m_b = []
      m_b.append(0)
      v_b = []
      v_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      if j==n-1:
        calculate_loss(y_hat, Y)
        #wandb.log({"loss": avg_loss, "epoch": epoch})
      grad_W, grad_b = backrop(X, Y, y_hat, W, a, h, act)[0:2]
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for j in range(1,l+1):
        m_w.append(np.zeros(W[j].shape))
        v_w.append(np.zeros(W[j].shape))
        m_b.append(np.zeros(b[j].shape))
        v_b.append(np.zeros(b[j].shape))
        
        m_w[j] = (beta1*m_prev_w[j] + (1-beta1)*grad_W[j])/(1-np.power(beta1,(i+1)))
        v_w[j] = (beta2*v_prev_w[j] + (1-beta2)*np.square(grad_W[j]))/(1-(np.power(beta2,(i+1))))
        m_b[j] = (beta1*m_prev_b[j] + (1-beta1)*grad_b[j])/(1-np.power(beta1,(i+1)))
        v_b[j] = (beta2*v_prev_b[j] + (1-beta2)*np.square(grad_b[j]))/(1-np.power(beta2,(i+1)))
        if j==l:
          # print(grad_W[j][4])
          #print(np.sqrt(v_w[j][4]+epsilon))
          print(np.power(beta1,(i+1)))
        W_new.append((W[j] - (learn_rate*np.divide(m_w[j],(np.sqrt(v_w[j]+epsilon))))))
        b_new.append((b[j] - (learn_rate*np.divide(m_b[j],(np.sqrt(v_b[j]+epsilon))))))
        m_prev_w[j] = m_w[j]
        v_prev_w[j] = v_w[j]
        m_prev_b[j] = m_b[j]
        v_prev_b[j] = v_b[j]
      del m_w
      del v_w
      del m_b
      del v_b
      W = W_new
      b = b_new

  return W, b

In [145]:
W, b = adam(10,0.001,0.9,0.99, 10**-8,512,'sigmoid','random',N)

0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
0.9
815.4000613856464
0.9
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: overflow encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: RuntimeWarning: overflow encountered in true_divide


0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
0.81
815.3990657356013
0.81
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0.7290000000000001
0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: overflow encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: RuntimeWarning: overflow encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in true_divide


0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
0.531441
nan
0.531441
0.4782969000000001
0.4782969000000001
0.4782969000000001
0.4782969000000001
0.

In [50]:
X_train[0,0:16]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [27]:
def nadam(epochs, learn_rate, beta1, beta2, epsilon, batch_size, act, init, N):
  W, b = initialize_weights(N,init)
  m_prev_w = []
  m_prev_w.append(0)
  m_prev_b = []
  m_prev_b.append(0)
  v_prev_w = []
  v_prev_w.append(0)
  v_prev_b = []
  v_prev_b.append(0)
  n=(np.floor(X_train.shape[1]/batch_size)+1).astype(int)
  for i in range(1,l+1):
    m_prev_w.append(np.zeros(W[i].shape))
    m_prev_b.append(np.zeros(b[i].shape))
    v_prev_w.append(np.zeros(W[i].shape))
    v_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      m_w = []
      m_w.append(0)
      v_w = []
      v_w.append(0)
      m_b = []
      m_b.append(0)
      v_b = []
      v_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      if j==n-1:
        calculate_loss(y_hat, Y)
        wandb.log({"loss": avg_loss, "epoch": epoch})
      #grad_W, grad_b = backrop(Y_train, y_hat, W, a, h, act)[0:2]
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for j in range(1,l+1):
        m_w.append(np.zeros(W[j].shape))
        v_w.append(np.zeros(W[j].shape))
        m_b.append(np.zeros(b[j].shape))
        v_b.append(np.zeros(b[j].shape))
        m_w[j] = (beta1*m_prev_w[j])/(1-(beta1**(i+1))) #+ (1-beta1)*grad_W[j])/(1-(beta1**(i+1)))
        v_w[j] = (beta2*v_prev_w[j])/(1-(beta2**(i+1))) #+ (1-beta2)*np.square(grad_W[j]))/(1-(beta2**(i+1)))
        m_b[j] = (beta1*m_prev_b[j])/(1-beta1**(i+1)) #+ (1-beta1)*grad_b[j])/(1-beta1**(i+1))
        v_b[j] = (beta2*v_prev_b[j])/(1-beta2**(i+1)) #+ (1-beta2)*np.square(grad_b[j]))/(1-beta2**(i+1))
        W_new.append((W[j] - (learn_rate*np.divide(m_w[j],(np.sqrt(v_w[j]+epsilon))))))
        b_new.append((b[j] - (learn_rate*np.divide(m_b[j],(np.sqrt(v_b[j]+epsilon))))))
      W = W_new
      b = b_new
      grad_W, grad_b = backrop(X, Y, y_hat, W, a, h, act)[0:2]
      for i in range(1,l+1):
        m_w[j] = (beta1*m_prev_w[j] + (1-beta1)*grad_W[j])/(1-(beta1**(i+1)))
        v_w[j] = (beta2*v_prev_w[j] + (1-beta2)*np.square(grad_W[j]))/(1-(beta2**(i+1)))
        m_b[j] = (beta1*m_prev_b[j] + (1-beta1)*grad_b[j])/(1-beta1**(i+1))
        v_b[j] = (beta2*v_prev_b[j] + (1-beta2)*np.square(grad_b[j]))/(1-beta2**(i+1))
        W_new.append((W[j] - (learn_rate*np.divide(m_w[j],(np.sqrt(v_w[j]+epsilon))))/batch_size))
        b_new.append((b[j] - (learn_rate*np.divide(m_b[j],(np.sqrt(v_b[j]+epsilon))))/batch_size))
        m_prev_w[j] = m_w[j]
        v_prev_w[j] = v_w[j]
        m_prev_b[j] = m_b[j]
        v_prev_b[j] = v_b[j]
      del m_w
      del v_w
      del m_b
      del v_b
      W = W_new
      b = b_new
  return W, b

In [ ]:
W,b = gradient_descent(10,0.001)

237097.25797024355
236504.65984101634
235915.35319173572
235329.34641918368
234746.64739842658
234167.26348595432
233591.20152293865
233018.46783869923
232449.06825427685
231883.00808620665


In [ ]:
W, b = nesterov_grad_descent(10, 0.001,0.9,1024)

2256.143703729159
2047.4780847643306
1911.0641613991295
1814.9768384497925
1743.8156035174752
1690.3848345853462
1650.2513357999287
1620.0728617417656
1597.1403145157337
1579.266565016062


In [57]:
W, b = rms_prop(3,0.001,0.9,10**-8,1024,'relu','random',N)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]] [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
nan


NameError: ignored

In [ ]:
W, b = nadam(10, 0.001,0.9,0.99,10**-8,128)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: overflow encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: RuntimeWarning: overflow encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: overflow encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: overflow encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: RuntimeWarning: overflow encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: overflow encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launc

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [ ]:
def grad_descent(W,b,grad_W,grad_b,learn_rate=0.001):
  W_new=[]
  W_new.append(0)
  b_new=[]
  b_new.append(0)
  m=X_train.shape[1]
  for i in range(1,l+1):
    #print(W[i].shape,grad_W[i].shape,b[i].shape,grad_b[i].shape)
    W_new.append(W[i] - (learn_rate*grad_W[i])/m)
    b_new.append(b[i] - (learn_rate*grad_b[i])/m)
  return W_new,b_new


Training

In [ ]:
def training(N,l,epochs,batch_size,learn_rate,X_input,y, act):
  W , b = initialize_weights(N)
  n=(np.floor(X_input.shape[1]/batch_size)+1).astype(int)
  for j in range(epochs):
    for j in range(n):
      X=X_input[:,[j*batch_size,(j+1)*batch_size]]
      y=y[:,[j*batch_size,(j+1)*batch_size]]
      a , h , y_hat = feedforward(X, W, b, N, l, act) 
      grad_W , grad_b = backrop(y,y_hat,W,a,h, act)[0:2]
      W , b = grad_descent(W,b,grad_W,grad_b,learn_rate)
    
    calculate_loss(y_hat,y)
  
  return W,b


In [ ]:
def predict(W,b,X_test,N,l, act):
  y_hat = feedforward(X_test, W, b, N, l, act)[2]
  y_class = np.argmax(y_hat,axis=0).reshape([1,X_test.shape[1]])

  return y_class

In [ ]:
Y_predict = predict(W_new,b_new,X_train,N,l)
print (Y_predict[0,0:4])
print (Y_test[0,0:4])
#sklearn.metrics.accuracy_score(Y_test, Y_predict)

[6 6 7 7]
[9 2 1 1]


In [ ]:
sweep_config = {
    'method': 'grid'
    }

In [ ]:
parameters_dict = {
    # 'l': {
    #     'values': [3, 4, 5]
    #   },
    # 'size of each layer': {
    #     'values': [32, 64, 128]
    #   },
    'weight decay': {
        'values': [0, 0.0005, 0.5]
      },
    'learn_rate': {
        'values': [1e-3,1e-4]
      },
    'epochs': {
        'values': [5,10]
      },
    'batch_size': {
        'values': [16,32,64]
      },
    'init': {
        'values': ['random', 'xavier']
      },
    'optimizer': {
        'values': ['sgd', 'mom', 'nest', 'rms', 'adam', 'nadam']
      },
    'act': {
          'values': ['sigmoid', 'tanh', 'ReLU']
      }
    }

sweep_config['parameters'] = parameters_dict

In [ ]:
import pprint

pprint.pprint(sweep_config)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="assignment 1")

In [ ]:
wandb.agent(sweep_id, train)

In [ ]:
def training_sweep(config=None):
  
  # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        if config.optimizer == 'sgd':
          gradient_descent(config.epochs, config.learn_rate, config.batch_size, config.act, config.init, N)
        if config.optimizer == 'mom':
          mom_grad_descent(config.epochs, config.learn_rate, gamma=0.9, config.batch_size, config.act, config.init, N)
        if config.optimizer == 'nest':
          nesterov_grad_descent(config.epochs, config.learn_rate, gamma=0.9, config.batch_size, config.act, config.init, N)
        if config.optimizer == 'rms':
          rms_prop(config.epochs, config.learn_rate, beta=0.9, epsilon=10**-8,config.batch_size, config.act, config.init, N)
        if config.optimizer == 'adam':
          adam(config.epochs, config.learn_rate, beta1=0.9, beta2=0.99, epsilon=10**-8,config.batch_size, config.act, config.init, N)
        if config.optimizer == 'nadam':
          nadam(config.epochs, config.learn_rate, beta1=0.9, beta2=0.99, epsilon=10**-8,config.batch_size, config.act, config.init, N)

        #wandb.log({"loss": avg_loss, "epoch": epoch})